## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/50_Startups.csv"
file_type = "csv"

# CSV options
df = spark.read.csv(file_location, header=True, inferSchema = True)


In [0]:
df.show()

+---------+--------------+---------------+----------+---------+
|R&D Spend|Administration|Marketing Spend|     State|   Profit|
+---------+--------------+---------------+----------+---------+
| 165349.2|      136897.8|       471784.1|  New York|192261.83|
| 162597.7|     151377.59|      443898.53|California|191792.06|
|153441.51|     101145.55|      407934.54|   Florida|191050.39|
|144372.41|     118671.85|      383199.62|  New York|182901.99|
|142107.34|      91391.77|      366168.42|   Florida|166187.94|
| 131876.9|      99814.71|      362861.36|  New York|156991.12|
|134615.46|     147198.87|      127716.82|California|156122.51|
|130298.13|     145530.06|      323876.68|   Florida| 155752.6|
|120542.52|     148718.95|      311613.29|  New York|152211.77|
|123334.88|     108679.17|      304981.62|California|149759.96|
|101913.08|     110594.11|      229160.95|   Florida|146121.95|
|100671.96|      91790.61|      249744.55|California| 144259.4|
| 93863.75|     127320.38|      249839.4

In [0]:
df.select('State').show()

+----------+
|     State|
+----------+
|  New York|
|California|
|   Florida|
|  New York|
|   Florida|
|  New York|
|California|
|   Florida|
|  New York|
|California|
|   Florida|
|California|
|   Florida|
|California|
|   Florida|
|  New York|
|California|
|  New York|
|   Florida|
|  New York|
+----------+
only showing top 20 rows



In [0]:
df.printSchema()

root
 |-- R&D Spend: double (nullable = true)
 |-- Administration: double (nullable = true)
 |-- Marketing Spend: double (nullable = true)
 |-- State: string (nullable = true)
 |-- Profit: double (nullable = true)



In [0]:
df.describe().show()

+-------+-----------------+------------------+------------------+----------+------------------+
|summary|        R&D Spend|    Administration|   Marketing Spend|     State|            Profit|
+-------+-----------------+------------------+------------------+----------+------------------+
|  count|               50|                50|                50|        50|                50|
|   mean|73721.61559999999|121344.63959999995|211025.09780000005|      null|112012.63920000002|
| stddev|45902.25648230754|28017.802755488683|122290.31072584528|      null|40306.180337650534|
|    min|              0.0|          51283.14|               0.0|California|           14681.4|
|    max|         165349.2|         182645.56|          471784.1|  New York|         192261.83|
+-------+-----------------+------------------+------------------+----------+------------------+



In [0]:
##handle categorial features
#state

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol='State', outputCol='State_indexed')
df_r = indexer.fit(df).transform(df)
df_r.show()

+---------+--------------+---------------+----------+---------+-------------+
|R&D Spend|Administration|Marketing Spend|     State|   Profit|State_indexed|
+---------+--------------+---------------+----------+---------+-------------+
| 165349.2|      136897.8|       471784.1|  New York|192261.83|          1.0|
| 162597.7|     151377.59|      443898.53|California|191792.06|          0.0|
|153441.51|     101145.55|      407934.54|   Florida|191050.39|          2.0|
|144372.41|     118671.85|      383199.62|  New York|182901.99|          1.0|
|142107.34|      91391.77|      366168.42|   Florida|166187.94|          2.0|
| 131876.9|      99814.71|      362861.36|  New York|156991.12|          1.0|
|134615.46|     147198.87|      127716.82|California|156122.51|          0.0|
|130298.13|     145530.06|      323876.68|   Florida| 155752.6|          2.0|
|120542.52|     148718.95|      311613.29|  New York|152211.77|          1.0|
|123334.88|     108679.17|      304981.62|California|149759.96| 

In [0]:
df_r.columns

Out[12]: ['R&D Spend',
 'Administration',
 'Marketing Spend',
 'State',
 'Profit',
 'State_indexed']

In [0]:

from pyspark.ml.feature import VectorAssembler
feature_assmbler = VectorAssembler(inputCols=['R&D Spend','Administration','Marketing Spend','State_indexed'], outputCol="Independent_features")
output = feature_assmbler.transform(df_r)

In [0]:
output.select('Independent_features').show()

+--------------------+
|Independent_features|
+--------------------+
|[165349.2,136897....|
|[162597.7,151377....|
|[153441.51,101145...|
|[144372.41,118671...|
|[142107.34,91391....|
|[131876.9,99814.7...|
|[134615.46,147198...|
|[130298.13,145530...|
|[120542.52,148718...|
|[123334.88,108679...|
|[101913.08,110594...|
|[100671.96,91790....|
|[93863.75,127320....|
|[91992.39,135495....|
|[119943.24,156547...|
|[114523.61,122616...|
|[78013.11,121597....|
|[94657.16,145077....|
|[91749.16,114175....|
|[86419.7,153514.1...|
+--------------------+
only showing top 20 rows



In [0]:
finalized_data = output.select('Independent_features','Profit')

In [0]:
finalized_data.show()

+--------------------+---------+
|Independent_features|   Profit|
+--------------------+---------+
|[165349.2,136897....|192261.83|
|[162597.7,151377....|191792.06|
|[153441.51,101145...|191050.39|
|[144372.41,118671...|182901.99|
|[142107.34,91391....|166187.94|
|[131876.9,99814.7...|156991.12|
|[134615.46,147198...|156122.51|
|[130298.13,145530...| 155752.6|
|[120542.52,148718...|152211.77|
|[123334.88,108679...|149759.96|
|[101913.08,110594...|146121.95|
|[100671.96,91790....| 144259.4|
|[93863.75,127320....|141585.52|
|[91992.39,135495....|134307.35|
|[119943.24,156547...|132602.65|
|[114523.61,122616...|129917.04|
|[78013.11,121597....|126992.93|
|[94657.16,145077....|125370.37|
|[91749.16,114175....| 124266.9|
|[86419.7,153514.1...|122776.86|
+--------------------+---------+
only showing top 20 rows



In [0]:
from pyspark.ml.regression import LinearRegression
#traintestsplit
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent_features', labelCol='Profit')
regressor=regressor.fit(train_data)

In [0]:
regressor.coefficients


Out[24]: DenseVector([0.7457, -0.0635, 0.0262, -206.1412])

In [0]:
regressor.intercept

Out[25]: 60671.68680991056

In [0]:
### Predictions
pred_results=regressor.evaluate(test_data)

In [0]:
## Final comparison
pred_results.predictions.show()

+--------------------+---------+------------------+
|Independent_features|   Profit|        prediction|
+--------------------+---------+------------------+
|[0.0,116983.8,451...|  14681.4| 54424.72403721338|
|[542.05,51743.15,...| 35673.41|57582.795498059946|
|[1315.46,115816.2...| 49490.75|61672.974187467895|
|[23640.93,96189.6...| 71498.49| 76070.32514850162|
|[28754.33,118546....| 78239.91| 79113.20545263261|
|[55493.95,103057....| 96778.92|100721.11990039248|
|[72107.6,127864.5...|105008.31|115372.47066847234|
|[76253.86,113867....|118474.03|118130.18701323066|
|[78013.11,121597....|126992.93| 118051.2095972497|
|[93863.75,127320....|141585.52| 128714.6459104969|
|[123334.88,108679...|149759.96| 153733.0241985913|
|[162597.7,151377....|191792.06| 183940.5038761621|
|[165349.2,136897....|192261.83|187437.04434747156|
+--------------------+---------+------------------+



In [0]:
## PErformance Metrics
pred_results.r2,pred_results.meanAbsoluteError,pred_results.meanSquaredError

Out[28]: (0.9272531699504732, 10184.020642055431, 207617182.57273838)